# Final project - Pedro Figueiras

In [43]:
import pandas as pd

In [44]:
# Getting the data about air quality stations (AQS) from Madrid city's webpage:
df = pd.read_excel("https://datos.madrid.es/egob/catalogo/212629-0-estaciones-control-aire.xls")
# Creating a new DF with the columns that I'm going to use for using K-means method:
df2 = df [['LATITUD', 'LONGITUD']]

In [45]:
# AQS maintenance will be performed by 4 service teams, I will use K-means for setting such number of clusters:
from sklearn.cluster import KMeans

# Set number of clusters
kclusters = 4

# Run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df2)

# Add K-means labels to the dataframe
k_means_labels = kmeans.labels_

# Adding the calculated K-means labels to our dataframe and sorting the columns:
df2.insert(0, 'SER TEAM', kmeans.labels_)

In [46]:
df2.loc[:, 'AQS'] = df['ESTACION'].values
df2 = df2[['AQS', 'SER TEAM', 'LATITUD', 'LONGITUD']]

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [47]:
# Sorting the values by Service Team (SER TEAM) and resetting the index:
df2 = df2.sort_values(by = 'SER TEAM')
df2.reset_index(drop=True, inplace=True)
df2

,AQS,SER TEAM,LATITUD,LONGITUD
0,Pza. de España,0,40.423882,-3.712257
1,Escuelas Aguirre,0,40.421553,-3.682316
2,Pza. Elíptica,0,40.385034,-3.718768
3,Parque del Retiro,0,40.414444,-3.682500
4,Villaverde,0,40.347147,-3.713317
5,Farolillo,0,40.394782,-3.731836
6,Casa de Campo,0,40.419358,-3.747345
7,Castellana,0,40.439890,-3.690373
8,Pza. del Carmen,0,40.419209,-3.703166
9,Mendez Alvaro,0,40.398099,-3.686814


In [53]:
# Using Foursquare, I will find the first recommended restaurant to have lunch nearby each AQS. 

import requests
from pandas.io.json import json_normalize

#These will be the arguments of the url/query (personal keys for Foursquare, Version, < 1000 meters, medium price and restaurant category):
CLIENT_ID = 'FBKRVQ4P2RGGNU1GVBPBHBU2XZ35SIOKH2ORFTSYIFMZ0GV3'
CLIENT_SECRET = 'LX3ET0IEA5EIY1O443EV0TFH010A4LGHZ2KNKPLM0XJPNGKD'
VERSION = '20190904'
radius = 1000
LIMIT = 1
prices = 2
cat = '4d4b7105d754a06374d81259'

#I get the recommended Restaurant for each AQS (using Foursquare API) and store all results in a new database "df3" using a loop:
cel = df2['AQS'].count()
df3 = pd.DataFrame()

for i in range (0, cel):
    latitude = (df2.loc[i]['LATITUD'])
    longitude = (df2.loc[i]['LONGITUD'])
    url = 'https://api.foursquare.com/v2/search/recommendations?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}&prices{}&'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, cat, radius, LIMIT, prices)
    results = requests.get(url).json()
    venues = results['response']['group']['results']
    dataframe = json_normalize(venues)
    dataframe = dataframe[['venue.name', 'venue.location.lat', 'venue.location.lng']]
    df3 = df3.append(dataframe, ignore_index=True)

In [54]:
#Some filtering on the new database to create the one to be used for showing the info in a map:
df3.columns = ['RESTAURANTE', 'LAT_R', 'LON_R']
df_map = pd.merge(df2, df3, left_index=True, right_index=True, how='outer')
df_map

,AQS,SER TEAM,LATITUD,LONGITUD,RESTAURANTE,LAT_R,LON_R
0,Pza. de España,0,40.423882,-3.712257,Club Allard,40.423028,-3.713987
1,Escuelas Aguirre,0,40.421553,-3.682316,Kabuki Wellington,40.421891,-3.684217
2,Pza. Elíptica,0,40.385034,-3.718768,La Peña Soriana,40.382060,-3.711332
3,Parque del Retiro,0,40.414444,-3.682500,Levél Veggie Bistro,40.415730,-3.677641
4,Villaverde,0,40.347147,-3.713317,Restaurante la Mancheguita 1978,40.342981,-3.711628
5,Farolillo,0,40.394782,-3.731836,Döner Kebap Nemrut,40.388257,-3.730849
6,Casa de Campo,0,40.419358,-3.747345,Buffet Libre Gran Avenida,40.411870,-3.750569
7,Castellana,0,40.439890,-3.690373,Santceloni,40.438891,-3.691322
8,Pza. del Carmen,0,40.419209,-3.703166,TAKOS,40.418938,-3.703748
9,Mendez Alvaro,0,40.398099,-3.686814,Magasand Deli,40.396811,-3.691293


In [50]:
# library to handle data in a vectorized manner
import numpy as np 
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# Download and import the map rendering library
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

# All requested packages already installed.



In [59]:
# create map of Madrid
map_clusters = folium.Map(location=[40.4167, -3.70325], tiles="Stamen Toner", zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map (circles) showing the AQS. Service teams have been defined by areas (different colors) via K-means:
markers_colors = []
for lat, lon, poi, cluster in zip(df_map['LATITUD'], df_map['LONGITUD'], df_map['AQS'], df_map['SER TEAM']):
    label = folium.Popup(str(poi) + ' - Area: ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
    
# add marker for the restaurant localtions. As it can be seen, each of them is very close to the AQS in the area:       
for la, lo, po, cluste in zip(df_map['LAT_R'], df_map['LON_R'], df_map['RESTAURANTE'], df_map['SER TEAM']):
    label = folium.Popup(str(po) + ' - Area: ' + str(cluste), parse_html=True)
    folium.Marker(        
        [la, lo],
        popup=label,
        icon=folium.Icon()).add_to(map_clusters) 
        
map_clusters